In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS, Chroma
from langchain_classic.storage import LocalFileStore

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    callbacks=[StreamingStdOutCallbackHandler()],
    streaming=True,
)

In [2]:
loader = TextLoader("./document.txt")

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=600,
    chunk_overlap=100,
    add_start_index=True,
)

docs = loader.load_and_split(text_splitter=text_splitter)

print(f"Number of documents: {len(docs)}")

Number of documents: 37


In [3]:
import hashlib

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
    key_encoder=lambda text: hashlib.sha256(text.encode("utf-8")).hexdigest(),
)

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=cached_embeddings,
)

retriever = vector_store.as_retriever()

In [4]:
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history",
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the context. If you don't know the answer, just say you don't know; don't make it up.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["history"]


chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)

C:\Users\trace\AppData\Local\Temp\ipykernel_11088\3384409080.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [5]:
def invoke_chain(question):
    result = chain.invoke(question)
    answer = result.content
    memory.save_context(
        {"input": question},
        {"output": answer},
    )
    print(answer)

In [6]:
invoke_chain("Is Aaronson guilty?")

Is Aaronson guilty?
According to the context, Aaronson is guilty of the crimes he was charged with.According to the context, Aaronson is guilty of the crimes he was charged with.


In [7]:
invoke_chain("What message did he write in the table?")

What message did he write in the table?
He wrote "I sold you and you sold me" on the table under the spreading chestnut tree.He wrote "I sold you and you sold me" on the table under the spreading chestnut tree.


In [8]:
invoke_chain("Who is Julia?")

Who is Julia?
Julia is a character with whom Winston has a romantic relationship. She is someone he loves and is deeply connected to, despite the oppressive regime they live under.Julia is a character with whom Winston has a romantic relationship. She is someone he loves and is deeply connected to, despite the oppressive regime they live under.
